In [1]:
import json
from jupyter_jsmol import JsmolView
from ipywidgets import Layout, widgets, interact
from IPython.display import display
import os

# An optional utility to display a progress bar
# for long-running loops. `pip install tqdm`.
from tqdm import tqdm
#import atomate2.common.schemas.symmetry as acss
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from jupyter_jsmol.pymatgen import quick_view

mpr = MPRester(api_key="XLgeaWrG6lipMAIH")

data = []

entrie = mpr.query("mp-924128", ["material_id", "cif"])
data.extend(entrie)

# Télécharger le fichier .cif

In [2]:
if not os.path.exists('lmapr1492_cifs'):
    os.mkdir('lmapr1492_cifs')

for d in tqdm(data):
    with open("lmapr1492_cifs/{}.cif".format(d["material_id"]), 'w') as f:
        f.write(d["cif"])

100%|██████████| 1/1 [00:00<00:00, 152.20it/s]


# Étude de la symmétrie

In [3]:
molecule = mpr.query("mp-924128", properties=["pretty_formula", "spacegroup.symbol", ])
cell_type = molecule[0]["spacegroup.symbol"]
print(molecule[0]["pretty_formula"], "\t \t cell type :", cell_type)

HfNiSn 	 	 cell type : F-43m


In [120]:
structure = mpr.get_structure_by_material_id("mp-924128")

struct = SpacegroupAnalyzer(structure)
struct_conv = struct.get_conventional_standard_structure()
#récupération de la structure conventionnel qui correspond à la maille du fichier CIF symmetrized
struct_analyze = SpacegroupAnalyzer(struct_conv)

view = quick_view(structure, conventional=True, supercell=[2,2,2])
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [95]:
# récupération des opération de symmétrie
sym_op = struct_analyze.get_symmetry_operations()
for i in range(len(sym_op)):
    print("----", i, "----\n", sym_op[i])
"""
print(sym_op[24])#translation (tau [0.5 0.5 0])
print("__________")
print(sym_op[14]) #rot C3 [1-1-1]
print("__________")
print(sym_op[34]) #rot C3 [-11-1] + translation (tau [0.5 0.5 0])
"""
#récupération de tous les sites d'atome dans la maille cristalline
sites = struct_conv.sites 
print("\n",sites)

---- 0 ----
 Rot:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
tau
[0. 0. 0.]
---- 1 ----
 Rot:
[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
---- 2 ----
 Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
---- 3 ----
 Rot:
[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
---- 4 ----
 Rot:
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
---- 5 ----
 Rot:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
tau
[0. 0. 0.]
---- 6 ----
 Rot:
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
---- 7 ----
 Rot:
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
---- 8 ----
 Rot:
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
tau
[0. 0. 0.]
---- 9 ----
 Rot:
[[ 0.  0. -1.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]
tau
[0. 0. 0.]
---- 10 ----
 Rot:
[[ 0.  0.  1.]
 [-1.  0.  0.]
 [ 0. -1.  0.]]
tau
[0. 0. 0.]
---- 11 ----
 Rot:
[[ 0.  0. -1.]
 [ 0. -1.  0.]
 [ 1.  0.  0.]]
tau
[0. 0. 0.]
---- 12 ----
 Rot:
[[ 0.  0. -1.]
 [ 1.  0.  0.]
 [ 0. -1.  0.]]
tau
[0. 0. 0

# Premier élément de symétrie
Le premier élément de symétrie est une rotation d'ordre 3 dans la direction $<1 \bar{1} \bar{1}>$.

La matrice qui représente cette opération de symétrie est la suivante : $$\left({\begin{array}{cc}
    0 & 0 & -1 \\
    -1 & 0 & 0 \\
    0 & 1 & 0 \\
  \end{array} }\right)$$
  
Cet élément de symétrie va être appliqué à l'atome $\textit{Hf}$ dont les coordonnées réduites sont : $(0.5, 0 , 0.5)$

In [113]:
op1 = sym_op[14]
print(op1)
hf1 = sites[2]
atome = str(hf1.species)

print("\nCoordonnées réduites avant l'opération de symétrie sur l'atome de", atome[:2], " : ", hf1.frac_coords)
# application de l'opérateur de symétrie
hf1_afterop = op1.operate(hf1.frac_coords)
print("Coordonnées réduites après l'opération :", hf1_afterop)

Rot:
[[ 0.  0. -1.]
 [-1.  0.  0.]
 [ 0.  1.  0.]]
tau
[0. 0. 0.]

Coordonnées réduites avant l'opération de symétrie sur l'atome de Hf  :  [0.5 0.  0.5]
Coordonnées réduites après l'opération : [-0.5 -0.5  0. ]


In [114]:
# représentation graphique
view = quick_view(struct_conv,supercell=[1,1,1])
display(view)
view.script('draw SYMOP 15 {atomno = 32}; select atomno=32; label on')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

# Deuxième élément de symétrie
Le deuxième élément de symétrie est une translation dans la direction $<1 1 0>$. 

La matrice qui représente cette opération de symétrie est la matrice identité : $$\begin{pmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 1
  \end{pmatrix}$$ à laquelle on ajoute une translation $(0.5, 0.5, 0)$
  
L'équation est donc : $$\begin{pmatrix}  1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 1 \end{pmatrix} 
\begin{pmatrix}  
x \\ 
y \\ 
z 
\end{pmatrix} + \begin{pmatrix}
    0.5 \\
    0.5 \\
    0 \\
  \end{pmatrix} = \begin{pmatrix}
   x+0.5 \\
   y+0.5 \\
   z \\
  \end{pmatrix}$$
  
Cet élément de symétrie va être appliqué à l'atome de $\textit{Ni}$ dont les coordonnées réduites sont : $(0.25 , 0.25, 0.75)$

In [115]:
op2 = sym_op[24] #rot C2 [011] * refl axe z
print(op2)
ni1 = sites[4]
atome = str(ni1.species)

print("\nCoordonnées réduites avant l'opération de symétrie sur l'atome de", atome[:2], " : ", ni1.frac_coords)
# application de l'opérateur de symétrie
ni1_afterop = op2.operate(ni1.frac_coords)
print("Coordonnées réduites après l'opération :", ni1_afterop)

Rot:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
tau
[0.5 0.5 0. ]

Coordonnées réduites avant l'opération de symétrie sur l'atome de Ni  :  [0.25 0.25 0.75]
Coordonnées réduites après l'opération : [0.75 0.75 0.75]


In [116]:
# représentation 3D graphique de l'opération
view = quick_view(struct_conv,supercell=[1,1,1])
display(view)
view.script('draw SYMOP 25 {atomno = 2}; select atomno=2; label on')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

# Troisième élément de symétrie
Le deuxième élément de symétrie est une rotation hélicoïdale $3_1$ dans la direction $<\bar{1} 1 \bar{1}>$. 

La matrice qui représente la rotation de cette opération de symétrie est la suivante : $$\begin{pmatrix}
    0 & 0 & 1 \\
    -1 & 0 & 0 \\
    0 & -1 & 0
  \end{pmatrix}$$
Pour compléter l'opération de symmétrie, il faut appliquer une translation $(0.5, 0.5, 0)$.
  
L'équation est donc : $$\begin{pmatrix}  0 & 0 & 1 \\
    -1 & 0 & 0 \\
    0 & -1 & 0 \end{pmatrix} 
\begin{pmatrix}  
x \\ 
y \\ 
z 
\end{pmatrix} + \begin{pmatrix}
    0.5 \\
    0.5 \\
    0 \\
  \end{pmatrix} = \begin{pmatrix}
   z+0.5 \\
   -x+0.5 \\
   -y \\
  \end{pmatrix}$$
  
Cet élément de symétrie va être appliqué à l'atome de potassium dont les coordonnées réduites sont : $(0.5 , 0.5, 0.5)$

In [117]:
op3 = sym_op[34] #rot C4 [010] * inv
print(op3)
sn1 = sites[9]
atome = str(sn1.species)

print("\nCoordonnées réduites avant l'opération de symétrie sur l'atome de", atome[:2], " : ", sn1.frac_coords)
# application de l'opérateur de symétrie
sn1_afterop = op3.operate(sn1.frac_coords)
print("Coordonnées réduites après l'opération :", sn1_afterop)

Rot:
[[ 0.  0.  1.]
 [-1.  0.  0.]
 [ 0. -1.  0.]]
tau
[0.5 0.5 0. ]

Coordonnées réduites avant l'opération de symétrie sur l'atome de Sn  :  [0.5 0.5 0.5]
Coordonnées réduites après l'opération : [ 1.   0.  -0.5]


In [119]:
# représentation 3D graphique de l'opération
view = quick_view(struct_conv,supercell=[1,1,1])
display(view)
view.script('draw SYMOP 35 {atomno = 13}; select atomno=13; label on')

JsmolView(layout=Layout(align_self='stretch', height='400px'))